# Testing single node datasets

Purpose: until now dataset represented by combination of tree types - now test for single node type

## Importing

In [1]:
import sys
import os

# Construct the path to the parent directory of the parent directory
parent_of_parent_dir = os.path.abspath(os.path.join(os.path.pardir, os.path.pardir))

# Add the parent directory of the parent directory to the system path
sys.path.insert(0, parent_of_parent_dir)

In [2]:
import optuna
import numpy as np
from sklearn.metrics import r2_score
import torch
from torch.utils.data import random_split, DataLoader

from etnn import TreeNode
from etnn.tools.training_tools import ConfigStore
from etnn.data.ferris_wheel import load_pure_ferris_wheel_dataset_single_node
from etnn.tools.training import train_epoch, eval_epoch
from etnn.nn.layer_framework import LayerManagementFramework
from etnn.tools.training_tools import seeding_all
from etnn.routines.run_config import choice_trainloader, choice_loss, choice_optim

## Control constants

In [3]:
dataset_path = "../../datasets/"
test_perc = 0.3
val_perc = 0.21
stability_count = 5
label = "tree-advanced" # although irrelevant

## Defining parameter search

In [4]:
def objective(trial):
    # init default config
    config = ConfigStore(
       in_dim=15,
       hidden_dim=trial.suggest_int("hidden_dim", 16, 512, step=16),
       out_dim=1,
       k=trial.suggest_int("k", 1, 5),
       dataset=-1 if normalized else 0,
       ds_size=10_000,
       num_gondolas=-1,
       num_part_pg=-1,
       loss_name='mse',
       optimizer_name='adam',
       num_max_epochs=30, # real: 100
       learning_rate=trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True),
       batch_size=1024,
       early_stop_tol=5,
       use_equal_batcher=trial.suggest_categorical("batcher", [True, False]),
       seed=420,
       label_type=label,
       final_label_factor=5/1000
    )
    # config = ConfigStore(
    #     in_dim=15,
    #     hidden_dim=16,
    #     out_dim=1,
    #     k=2,
    #     dataset=-1 if normalized else 0,
    #     ds_size=10_000,
    #     num_gondolas=-1,
    #     num_part_pg=-1,
    #     loss_name='mse',
    #     optimizer_name='adam',
    #     num_max_epochs=30, # real: 100
    #     learning_rate=0.002,
    #     batch_size=1024,
    #     early_stop_tol=5,
    #     use_equal_batcher=False,
    #     seed=420,
    #     label_type=label,
    #     final_label_factor=5/1000
    # )

    # loading dataset
    dataset, df_index = load_pure_ferris_wheel_dataset_single_node(
        node_type=node_type,
        num_elem=size_elem,
        num_to_generate=config.ds_size,
        dataset_path=dataset_path,
        final_label_factor=config.final_label_factor,
        normalize=True if config.dataset == -1 else 0
    )
    # splitting off test dataset
    generator = torch.Generator().manual_seed(config.seed)
    train_ds, val_ds, _ = random_split(
        dataset,
        [1 - test_perc - val_perc, val_perc, test_perc],
        generator=generator
    )

    # loaders
    train_loader = choice_trainloader(config, df_index, train_ds)
    val_loader = DataLoader(val_ds, batch_size=4 * config.batch_size, shuffle=False)

    # define device
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # set seed for reproducability
    seeding_all(config.seed)

    # define model
    model = LayerManagementFramework(
        in_dim=config.in_dim,
        tree=TreeNode(node_type, [TreeNode("E", size_elem)]),
        hidden_dim=config.hidden_dim,
        out_dim=config.out_dim,
        k=config.k
    ).to(device)

    # learning tools
    criterion = choice_loss(config)
    optimizer = choice_optim(config, model)

    # init score list
    score_list = []

    # train for specified number of epochs
    for epoch in range(config.num_max_epochs):
        _, _, _ = train_epoch(
            model,
            train_loader,
            optimizer,
            device,
            criterion
        )

        _, val_true_y, val_pred_y = eval_epoch(
            model,
            val_loader,
            device,
            criterion
        )

        # calc r2 score and append
        score = r2_score(y_true=val_true_y, y_pred=val_pred_y)
        score_list += [score]
        trial.report(score, epoch)

    # calculate objective
    # display(score_list)
    # idea: last x r2 scores (why not last one? for stability purposes)
    obj = np.array(score_list)[-stability_count:]
    return np.mean(obj)

## Parameter search

### S

In [5]:
node_type = "S"
size_elem = 10
n_trials = 50
normalized = False
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-10 16:47:57,320] A new study created in memory with name: Study node type S, normalized: False


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2023-12-10 16:49:37,223] Trial 0 finished with value: -18.89985937637609 and parameters: {'hidden_dim': 32, 'k': 4, 'learning_rate': 1.969253832608162e-05, 'batcher': True}. Best is trial 0 with value: -18.89985937637609.
[I 2023-12-10 16:51:14,136] Trial 1 finished with value: -1.3926707514182173 and parameters: {'hidden_dim': 112, 'k': 5, 'learning_rate': 0.006367508635174888, 'batcher': True}. Best is trial 1 with value: -1.3926707514182173.
[I 2023-12-10 16:52:34,021] Trial 2 finished with value: -2.4964457239287503 and parameters: {'hidden_dim': 304, 'k': 1, 'learning_rate': 0.00707600879933757, 'batcher': False}. Best is trial 1 with value: -1.3926707514182173.
[I 2023-12-10 16:53:53,903] Trial 3 finished with value: -4.890314921822672 and parameters: {'hidden_dim': 192, 'k': 5, 'learning_rate': 0.0006683115876689624, 'batcher': True}. Best is trial 1 with value: -1.3926707514182173.
[I 2023-12-10 16:55:13,341] Trial 4 finished with value: 0.3169671388263199 and parameters: {'

In [6]:
study.trials_dataframe().to_csv(f"single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}.csv")

In [7]:
node_type = "S"
size_elem = 10
n_trials = 50
normalized = True
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-10 18:06:34,949] A new study created in memory with name: Study node type S, normalized: True


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2023-12-10 18:08:07,797] Trial 0 finished with value: 0.8567521584625112 and parameters: {'hidden_dim': 480, 'k': 4, 'learning_rate': 8.412678798122184e-05, 'batcher': False}. Best is trial 0 with value: 0.8567521584625112.
[I 2023-12-10 18:09:36,946] Trial 1 finished with value: -35.5024216842971 and parameters: {'hidden_dim': 64, 'k': 5, 'learning_rate': 6.193045088115752e-05, 'batcher': False}. Best is trial 0 with value: 0.8567521584625112.
[I 2023-12-10 18:11:11,363] Trial 2 finished with value: 0.9234221030401196 and parameters: {'hidden_dim': 496, 'k': 2, 'learning_rate': 0.0003961604506306014, 'batcher': True}. Best is trial 2 with value: 0.9234221030401196.
[I 2023-12-10 18:12:44,196] Trial 3 finished with value: 0.9305275057068233 and parameters: {'hidden_dim': 384, 'k': 5, 'learning_rate': 0.005564500727225513, 'batcher': False}. Best is trial 3 with value: 0.9305275057068233.
[I 2023-12-10 18:13:59,322] Trial 4 finished with value: 0.9360525732990842 and parameters: {'hi

In [8]:
study.trials_dataframe().to_csv(f"single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}.csv")

### Q

In [5]:
node_type = "Q"
size_elem = 10
n_trials = 50
normalized = False
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-10 19:07:31,641] A new study created in memory with name: Study node type Q, normalized: False


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2023-12-10 19:08:47,267] Trial 0 finished with value: 0.8290976079765999 and parameters: {'hidden_dim': 16, 'k': 1, 'learning_rate': 0.007434023341450371, 'batcher': False}. Best is trial 0 with value: 0.8290976079765999.
[I 2023-12-10 19:10:20,956] Trial 1 finished with value: 0.4488305277302955 and parameters: {'hidden_dim': 480, 'k': 2, 'learning_rate': 0.00032987220805189444, 'batcher': False}. Best is trial 0 with value: 0.8290976079765999.
[I 2023-12-10 19:11:51,062] Trial 2 finished with value: -852.1203689740853 and parameters: {'hidden_dim': 400, 'k': 4, 'learning_rate': 0.000988181284667888, 'batcher': True}. Best is trial 0 with value: 0.8290976079765999.
[I 2023-12-10 19:13:18,962] Trial 3 finished with value: 0.86154356641458 and parameters: {'hidden_dim': 224, 'k': 2, 'learning_rate': 0.003091213250859929, 'batcher': True}. Best is trial 3 with value: 0.86154356641458.
[I 2023-12-10 19:14:57,379] Trial 4 finished with value: -1.272604568095252 and parameters: {'hidden_

In [6]:
study.trials_dataframe().to_csv(f"single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}.csv")

In [7]:
node_type = "Q"
size_elem = 10
n_trials = 50
normalized = True
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-10 20:17:28,896] A new study created in memory with name: Study node type Q, normalized: True


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2023-12-10 20:18:33,036] Trial 0 finished with value: -119.43906665744025 and parameters: {'hidden_dim': 128, 'k': 1, 'learning_rate': 1.759628616499722e-05, 'batcher': False}. Best is trial 0 with value: -119.43906665744025.
[I 2023-12-10 20:19:36,809] Trial 1 finished with value: 0.40900534939636285 and parameters: {'hidden_dim': 80, 'k': 1, 'learning_rate': 0.00011521670378851365, 'batcher': False}. Best is trial 1 with value: 0.40900534939636285.
[I 2023-12-10 20:20:49,687] Trial 2 finished with value: 0.8157997990944853 and parameters: {'hidden_dim': 480, 'k': 3, 'learning_rate': 2.625756262108787e-05, 'batcher': True}. Best is trial 2 with value: 0.8157997990944853.
[I 2023-12-10 20:21:52,999] Trial 3 finished with value: 0.6683089275144458 and parameters: {'hidden_dim': 48, 'k': 1, 'learning_rate': 0.001565311864553082, 'batcher': False}. Best is trial 2 with value: 0.8157997990944853.
[I 2023-12-10 20:23:04,947] Trial 4 finished with value: 0.7921733818371012 and parameters:

In [8]:
study.trials_dataframe().to_csv(f"single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}.csv")

### C

In [ ]:
node_type = "C"
size_elem = 10
n_trials = 50
normalized = False
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

[I 2023-12-10 21:21:14,299] A new study created in memory with name: Study node type C, normalized: False


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2023-12-10 21:22:56,591] Trial 0 finished with value: -93.62119261596906 and parameters: {'hidden_dim': 416, 'k': 2, 'learning_rate': 0.00021010255437875497, 'batcher': False}. Best is trial 0 with value: -93.62119261596906.
[I 2023-12-10 21:24:38,087] Trial 1 finished with value: 0.9110765116333253 and parameters: {'hidden_dim': 512, 'k': 2, 'learning_rate': 0.00010861813988293573, 'batcher': True}. Best is trial 1 with value: 0.9110765116333253.
[I 2023-12-10 21:26:12,903] Trial 2 finished with value: 0.48718422116471416 and parameters: {'hidden_dim': 96, 'k': 2, 'learning_rate': 7.431742231261732e-05, 'batcher': True}. Best is trial 1 with value: 0.9110765116333253.
[I 2023-12-10 21:27:57,025] Trial 3 finished with value: 0.1606330510849237 and parameters: {'hidden_dim': 336, 'k': 5, 'learning_rate': 4.281482343262913e-05, 'batcher': False}. Best is trial 1 with value: 0.9110765116333253.
[I 2023-12-10 21:29:45,703] Trial 4 finished with value: -78.15832713991628 and parameters: 

In [ ]:
study.trials_dataframe().to_csv(f"single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}.csv")

In [ ]:
node_type = "C"
size_elem = 10
n_trials = 50
normalized = True
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

In [ ]:
study.trials_dataframe().to_csv(f"single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}.csv")

### P

In [ ]:
node_type = "P"
size_elem = 10
n_trials = 50
normalized = False
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

In [ ]:
study.trials_dataframe().to_csv(f"single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}.csv")

In [ ]:
node_type = "P"
size_elem = 10
n_trials = 50
normalized = True
study = optuna.create_study(study_name=f"Study node type {node_type}, normalized: {normalized}", directions=['maximize'])
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

In [ ]:
study.trials_dataframe().to_csv(f"single-node_t-{node_type}_n-{size_elem}_normalized-{normalized}.csv")